# Classifying COVID-19 (And Pneumonia) in Patient X-Rays

# Load libraries

In [ ]:
import random

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset, random_split

In [ ]:
import optuna

In [ ]:
from torchvision import models

## Obtain and Augment the Data

In [ ]:
# Dataset transformations
# These are for Resnet
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(90),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
# Image folders for the dataset
training_data = ImageFolder(root='dataset/train', transform=data_transforms['train'])
testing_data = ImageFolder(root='dataset/test', transform=data_transforms['val'])

In [ ]:
# Decide on the size of each subset of the dataset
train_size = int(0.75 * len(training_data))
val_size = int(0.25 * len(training_data))
test_size = len(testing_data)

print(train_size, val_size, len(training_data))

3858 1286 5144


In [ ]:
# Form the datasets
train_data, val_data = random_split(training_data, [train_size, val_size])
test_data = testing_data

print(f"Training set: {len(train_data)}, Validation set: {len(val_data)}, Testing set: {len(test_data)}")

Training set: 3858, Validation set: 1286, Testing set: 1288


In [ ]:
# Data loaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Resnet Model

In [ ]:
# Objective function for Optuna
def objective(trial):
    # Hyperparameter tuning
    learning_rate = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    l1_lambda = trial.suggest_float("l1_lambda", 1e-6, 1e-2, log=True)
    l2_lambda = trial.suggest_float("l2_lambda", 1e-6, 1e-2, log=True)

    # Resnet model
    model = models.resnet18(pretrained=True)

    # Change the last layer's output to 3 outputs
    for param in model.parameters():
        param.requires_grad = False
        # Replace the last fully-connected layer
        # Parameters of newly constructed modules have requires_grad=True by default
    model.fc = nn.Linear(512, 3) # Assume the fc7 layer has 512 neurons


    # Freeze layers to reduce number of parameters to update
    for name, param in model.named_parameters():
        if "fc" in name:  # Still want to keep classification layer
            param.requires_grad = True
        else:
            param.requires_grad = False

    # Define the optimizer with L2 regularization
    optimizer = optim.SGD(
        model.parameters(),
        lr=learning_rate,
        momentum=0.9,
        weight_decay=l2_lambda  # L2 regularization (also known as weight decay)
    )

    # Loss function with L1 regularization
    criterion = nn.CrossEntropyLoss()

    # Data Loaders (use the previously defined loaders)
    train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=64, shuffle=False)

    # Training loop
    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, (images, labels) in enumerate(train_loader, 0):
            images = images
            labels = labels

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)

            # L1 regularization
            l1_loss = 0
            for param in model.parameters():
                l1_loss += torch.sum(torch.abs(param))
            loss += l1_lambda * l1_loss

            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images
                labels = labels

                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        trial.report(val_loss, epoch)

        # Handle pruning based on the intermediate results.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return val_loss

In [ ]:
%%time

# Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=4)

# Best hyperparameters
print(f"Best hyperparameters: {study.best_trial.params}")

[I 2024-09-24 05:04:41,693] A new study created in memory with name: no-name-bc07b0af-f9a6-4e73-ada6-a9c4ff4fbb5a
[I 2024-09-24 06:55:05,123] Trial 0 finished with value: 0.4418339232603709 and parameters: {'lr': 0.0006154484155835949, 'l1_lambda': 5.413410501721802e-06, 'l2_lambda': 0.0002534862797240738}. Best is trial 0 with value: 0.4418339232603709.
[I 2024-09-24 08:24:05,172] Trial 1 finished with value: 0.6893243051710582 and parameters: {'lr': 3.947838285233008e-05, 'l1_lambda': 8.341768708114669e-05, 'l2_lambda': 0.0004617923174105667}. Best is trial 0 with value: 0.4418339232603709.
[I 2024-09-24 09:49:33,108] Trial 2 finished with value: 0.73722224292301 and parameters: {'lr': 2.6296239487246457e-05, 'l1_lambda': 7.812577652590432e-05, 'l2_lambda': 0.0022172055038564326}. Best is trial 0 with value: 0.4418339232603709.
[I 2024-09-24 11:21:20,644] Trial 3 finished with value: 0.3807932209400904 and parameters: {'lr': 0.0018564932880784544, 'l1_lambda': 1.737555960966359e-06, 

Best hyperparameters: {'lr': 0.0018564932880784544, 'l1_lambda': 1.737555960966359e-06, 'l2_lambda': 2.7745524655250464e-05}
Wall time: 6h 16min 39s


In [ ]:
# Best hyperparameters
print(f"Best val loss: {study.best_value}")


Best val loss: 0.3807932209400904


### References
1. The Module 25 office hour hyperparameter tuning notebook
2. (resnet model) https://github.com/AarohiSingla/Image-Classification-Using-Pytorch/blob/main/image_classification.ipynb